In [76]:
import pandas as pd
import numpy as np
import seaborn as sns

In [77]:
df_timereg =pd.read_csv('/home/jupyter-a.solovev-15/shared/problem1-reg_data.csv', sep = ';')
df_entry_time =pd.read_csv('/home/jupyter-a.solovev-15/shared/problem1-auth_data.csv', sep = ';')

In [78]:
# ограничение дат
# промежуток reg_date - ограничение когорт по времени регистрации
# промежуток auth_date - за какой временой промежуток будут показываться эти когорты
reg_date_start = "2020-06-01"
reg_date_end = "2020-06-30"
auth_date_start = "2020-06-01"
auth_date_end = "2020-06-30"

In [92]:
def retention(df_timereg, df_entry_time, reg_date_start, reg_date_end, auth_date_start, auth_date_end):
    # приведение у нужному понятному формату даты
    df_entry_time.auth_ts = pd.to_datetime(df_entry_time.auth_ts, unit='s').round("d")
    df_timereg.reg_ts = pd.to_datetime(df_timereg.reg_ts, unit='s').round("d")
    
    # Ограничение данных по запросу дат
    start_reg = pd.to_datetime(reg_date_start)
    end_reg = pd.to_datetime(reg_date_end)
    start_auth = pd.to_datetime(auth_date_start)
    end_auth = pd.to_datetime(auth_date_end)
    df_timereg= df_timereg.query('@start_reg <= reg_ts <= @end_reg')
    df_entry_time= df_entry_time.query('@start_auth <= auth_ts <= @end_auth')
    
    #Создание датафрейма со всеми датами
    day = pd.DataFrame(pd.date_range(start_auth, end_auth,freq='D'))
    day.columns = ['day']
    
    # создание датафрейма с датами существования каждого аккаунта
    df_timereg['key'] = 1
    day['key'] = 1
    template = df_timereg.merge(day,on='key')[['uid','reg_ts','day']]
    template = template[template['day'] >= template['reg_ts']]
    
    # создание датафрейма с обозначением регистрации, дней существования и дней посещения
    df_entry_time['key'] = 1
    dff = df_entry_time.rename(columns={'auth_ts': 'day'})
    report = template.merge(dff,on=['uid','day'],how='left')
    
    # перевод дат в лайфтайм после регистрации с обозначением активных дней
    report['active'] = (report['key'] > 0)*1
    report['lifetime'] = (report['day'] - report['reg_ts'])
    retention = report.pivot_table(index='reg_ts',columns='lifetime',values='active',aggfunc='mean').mul(100)

    
    return retention

In [93]:
retention = retention(df_timereg, df_entry_time, reg_date_start, reg_date_end, auth_date_start, auth_date_end)

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [94]:
def avg_retention(df_timereg, retention):
    

    # каждой дате сопоставим количество регистраций
    count_id_date = df_timereg.groupby('reg_ts', as_index = False).agg({'uid': 'count'})
    av_ret = retention.merge(count_id_date,on=['reg_ts'],how='left')
    # считаем средневзвешанный retention 
    columns = av_ret.columns.to_list()
    columns.remove('reg_ts')
    columns.remove('uid')
    for column in columns:
        av_ret[column] = av_ret[column] * av_ret['uid']

    av_ret.loc['weg_avg_ret', :] = round(av_ret.sum(axis=0) / av_ret['uid'].sum(), 2)
    # убираем ненужные данные
    av_ret = av_ret.drop(columns= ['reg_ts', 'uid']) 
    av_ret = av_ret.drop(av_ret.index[0:30])    
    return av_ret

In [95]:
avg_retention(df_timereg, retention)

,0 days,1 days,2 days,3 days,4 days,5 days,6 days,7 days,8 days,9 days,...,20 days,21 days,22 days,23 days,24 days,25 days,26 days,27 days,28 days,29 days
weg_avg_ret,100.0,1.87,3.68,4.24,4.53,4.89,5.59,4.36,3.53,3.53,...,1.37,1.14,0.94,0.8,0.77,0.53,0.41,0.29,0.18,0.1


In [96]:
retention

lifetime,0 days,1 days,2 days,3 days,4 days,5 days,6 days,7 days,8 days,9 days,...,20 days,21 days,22 days,23 days,24 days,25 days,26 days,27 days,28 days,29 days
reg_ts,,,,,,,,,,,,,,,,,,,,,
2020-06-01,100.0,1.688693,4.111601,4.992658,5.139501,6.901615,7.856094,5.726872,6.534508,3.597651,...,4.845815,4.258443,3.744493,3.817915,4.625551,3.377386,2.936858,3.303965,3.230543,3.157122
2020-06-02,100.0,2.346041,3.812317,4.032258,4.325513,5.351906,7.038123,4.838710,4.252199,4.398827,...,3.519062,3.665689,3.739003,3.225806,4.105572,2.565982,2.639296,2.199413,2.272727,NaN
2020-06-03,100.0,1.755669,4.754938,4.828091,4.462326,5.267008,7.315289,5.779078,5.120702,5.779078,...,4.462326,4.242868,4.242868,4.096562,3.950256,3.657644,3.803950,3.291880,NaN,NaN
2020-06-04,100.0,1.827485,4.020468,5.190058,3.947368,6.067251,6.725146,5.628655,5.043860,4.166667,...,3.508772,3.801170,3.289474,3.143275,4.093567,3.728070,3.070175,NaN,NaN,NaN
2020-06-05,100.0,1.896426,3.282276,4.376368,5.616338,4.959883,6.710430,5.470460,4.230489,5.032823,...,3.865791,3.719912,4.084610,3.646973,4.011670,2.990518,NaN,NaN,NaN,NaN
2020-06-06,100.0,1.966497,3.423161,4.661326,4.661326,4.952658,6.846322,4.369993,4.806992,4.369993,...,3.058995,3.641661,2.403496,3.423161,2.840495,NaN,NaN,NaN,NaN,NaN
2020-06-07,100.0,2.252907,3.633721,5.014535,5.014535,6.758721,7.194767,6.613372,4.941860,5.377907,...,5.014535,4.651163,3.851744,3.197674,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-08,100.0,1.670298,3.703704,4.793028,4.647785,5.809731,7.915759,4.865650,5.010893,5.374001,...,4.865650,2.977487,3.267974,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-09,100.0,1.521739,3.550725,4.855072,3.913043,5.507246,6.521739,5.942029,4.202899,4.492754,...,4.420290,3.768116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
